In [ ]:
s = str('c:\사진\2004년\2004년01월\2004년01월06일 신월성 사업팀 덕유산')
s.repalce('사진', '/')
print(s)

In [ ]:
import pandas as pd

df = pd.DataFrame({"Name": ["blue", 
                            "delta", 
                            "echo", 
                            "charlie", 
                            "alpha"], 
                    "Type": ["Raptors",
                            "Raptors",
                            "Raptors",
                            "Raptors",
                            "Tyrannosaurus rex"]
                    })

print (df.index[(df['Name'] == 'blue')].tolist())
print ('\n')
print (df.loc[df['Name'] == 'blue'])
print ('\n')
print (df.loc[(df['Name'] == 'charlie') & (df['Type'] =='Raptors')])

In [ ]:
import mysql
import mysql.connector
conn = mysql.connector.connect(host='localhost', port='3306', database='mypictures', user='root', password='1234')
cursor = conn.cursor(dictionary=True)
cursor.execute('select pictureFileSrcDir from mypicturefiles;')
result = cursor.fetchall()
#print(result['pictureFileSrcDir'])
for dirs in result:
    if dirs == None:
        pass
    else:
        print(dirs['pictureFileSrcDir'])
print()
cursor.execute('select pictureFileSrcDir from mypicturefiles;')
result = cursor.fetchone()
#print(result['pictureFileSrcDir'])
              
#print(result)
#excute.

#existDF = pd.DataFrame(index=range(0, 0), columns={'dirName', 'fileName'})
#print(existDF)

In [ ]:
def convert_size(size_bytes):
    import math
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 4)
    return "%s %s" % (s, size_name[i])
convert_size(152367)

In [ ]:
os.sep = '/'
#pathlib.Path.sep 
path = 'c:\\'
file = 'test.txt'
os.path.sep = '/'
os.path.sep
os.path.join(path, file)



In [ ]:
import os
import re
import pathlib
import stat
from datetime import datetime
from PIL import Image
from PIL.ExifTags import TAGS
from time import time
import time
import mysql
import mysql.connector
import math

#path = 'E:\\source\\pygame\\Nado Game\\pyqt5\\사진관리\\data'
path = 'D:\\개인자료\\개인 사진\\2004년\\041123 설악산 대청봉 041113'
file = '20060116_132458_DSCF2972.JPG'
#remark = 

ALLOW_GRAPHIC = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tif', 'tiff']
ALLOW_MEDIA = ['.avi','.mov', '.mp4']

def renameFolder(path, file, remark):
    if self.listWidget.currentItem() == None:
        QMessageBox.about(self, "경고", "파일이 선택되지 않았습니다. 파일을 선택해 주세요.")
        return
    file = self.listWidget.currentItem().text()
    path, f = os.path.split(file)
    root, lastDir = os.path.split(path)
    m = self.checkReMatch(lastDir)
    newDirName = lastDir[:m.end()] + remark
    newDir = pathlib.Path(root, newDirName)
    #path = os.path.abspath(path)
    print('renameFolder:', str(newDir), remark, path)
    self.updateRemarkDB(str(newDir), remark, str(path))
    #os.rename(path, newDir)
    #self.lineEdit.setText(str(newDir))
    #self.list_files()
    
def updateRemarkDB(newDir, remark, path):
    #newDir = str(newDir).replace('\\', '\\\\', -1)
    #path = str(path).replace('\\', '\\\\', -1)
    print('updateRemarkDB: ', newDir, remark, path)
    conn = connDB()
    cursor = conn.cursor(dictionary=True)
    sql = 'UPDATE mypicturefiles SET pictureFileDestDir = %s, remark = %s WHERE pictureFileDestDir = %s;'
    val = (str(newDir), remark, str(path))
    print('updateRemarkDB val: ', val)
    cursor.execute(sql, val)
    conn.commit()
    #print(cursor.rowcount, "record(s) affected")
    cursor.close()
    conn.close()
    print(cursor.rowcount, "record(s) affected")
    return  

def listFiles():
    return os.walk(path)

def get_remark(path):
    lastDir = os.path.basename(path)
    m = checkReMatch(lastDir)
    if m and len(lastDir) > m.end():
        #print(sub[m.end():])
        return lastDir[m.end():]
    else:
        return ""

def connDB():
    conn = mysql.connector.connect(
        host='localhost', 
        port='3306', 
        database='mypictures', 
        user='root', 
        password='1234'
    )
    return conn
def takePictureTime(path, f):
    filename = pathlib.Path(path, f)
    try :
        with Image.open(filename) as im:
            info = im._getexif()
            #print('info = im._getexif()', info)
        # 새로운 딕셔너리 생성
        taglabel = {}
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            taglabel[decoded] = value
        print(checkReMatch(taglabel['DateTimeOriginal']))
        print(taglabel['DateTimeOriginal'])
        #print('tag, value', tag, value)
        if checkReMatch(taglabel['DateTimeOriginal']):# != '    :  :     :  :  ': #taglabel['DateTimeOriginal'] != None or 
            s = taglabel['DateTimeOriginal']
            print(f, "s = taglabel['DateTimeOriginal']", s, s == '    :  :     :  :  ')
            return s
        elif checkReMatch(taglabel['DateTimeDigitized']):# != '    :  :     :  :  ':
            s = taglabel['DateTimeDigitized']
            print(f, "s = taglabel['DateTimeDigitized']", s)
            return s
        elif checkReMatch(taglabel['DateTime']):# != '    :  :     :  :  ':
            s = taglabel['DateTime']
            print(f, "s = taglabel['DateTime']", s)
            return s
        else:
            return None
        '''
        s = taglabel['DateTimeOriginal']
        #print('{taglabel}', taglabel)
        print(f,"s = taglabel['DateTimeOriginal']: ", s)
        timestamp = time.mktime(datetime.strptime(s, '%Y:%m:%d %H:%M:%S').timetuple())
        return s
        #return timestamp
        '''
    except:
        return None

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])


def fileToDB():
    Tfiles = listFiles()
    for path, subDir, files in Tfiles:
        #path = path.replace('\\', '/')
        for file in files: 
            takePictureTime(path, file)
            print('match file', checkReSearch(file))
            '''
            file_size = os.path.getsize(os.path.join(path, file))
            #print(path, file, convert_size(file_size))
            path = pathlib.Path(path)
            path = str(path)
            #p = str(path).replace('\\', '/')
            #print('pathlib: ', path)
            exist = selectDB([str(file)])
            #e = exist['pictureFileDestDir'].replace('\\', '/')
            #print(exist, file)
            #print(file == exist['pictureFileName'])
            print('path: ', path, "'exist['pictureFileDestDir']]:" , exist['pictureFileDestDir'], str(path) == exist['pictureFileDestDir'])
            print ('c:\사진정리\2006년\2006년01월\2006년01월16일 채혁입대' == 'c:\사진\2006년\2006년01월\2006년01월16일 채혁입대')
            '''
            '''
            remark = get_remark(path)
            tt = takePictureTime(path, file)
            exist = selectDB([str(file)])
            pathExist = selectPathDB([path])
            print('exist: ', exist)

            if not exist:
                print('not exist insertDB: ', path, file)
                insertDB(file, path, path, tt, remark)
            elif pathExist:
                pass
            else:
                dupliExist = selectDupliDB(file, path)
                #print(dupliExist)
                if not dupliExist:
                    print('not duplicated insertDupliDB: ', path, file)
                    insertDupliDB(file, path, path, remark)
            '''

            
            
def suffixVerify(path, f):
    # f = pathlib.Path(path, name)  # 원본 파일
    src = pathlib.Path(path, f)
    if src.suffix.lower() in ALLOW_GRAPHIC:
        return True
    elif src.suffix.lower() in ALLOW_MEDIA:
        if checkReMatch(f):
            return True
    else:
        return False

def checkReMatch(path):
    checker = re.compile(r'(19\d\d|20\d\d|\d\d)[년\-_.: ]?(0[1-9]|1[012])[월\-_.: ]?(0[1-9]|[12][0-9]|3[01])[일]?[_ ]([01][0-9]|2[0-3])[시\-_.: ]?([0-5][0-9])[분\-_.: ]?([0-5][0-9])[초]?')
    return checker.match(path)
def checkReSearch(path):
    checker = re.compile(r'(19\d\d|20\d\d|\d\d)[년\-_.: ]?(0[1-9]|1[012])[월\-_.: ]?(0[1-9]|[12][0-9]|3[01])[일]?[_ ]([01][0-9]|2[0-3])[시\-_.: ]?([0-5][0-9])[분\-_.: ]?([0-5][0-9])[초]?')
    return checker.search(path)

def updateRemarkDB(desDir, remark, basename):
    conn = connDB()
    cursor = conn.cursor(dictionary=True)
    sql = "UPDATE mypicturefiles (pictureSrcDir =%s, remark) \
    values (%s, %s) WHERE pictureSrcDir=%s"
    val = (desDir, remark, basename)
    cursor.execute(sql, val)
    conn.commit()
    conn.close()
    return

#UPDATE table_name SET name = '테스트 변경', country = '대한민국' WHERE id = 1105
def selectDupliDB(file, path):
    conn = connDB()
    #if conn.is_connected():
    #print(True)
    cursor = conn.cursor(dictionary=True)
    sql = "select * from duplicatedpictures \
        where duPicFileName = %s and srcDir = %s;"
    val = (file, path)
    cursor.execute(sql, val)
    #print(cursor)
    result = cursor.fetchall()
    conn.close()
    return result


def insertDupliDB(f, srcDir, desDir, d):
    print(d)
    conn = connDB()
    cursor = conn.cursor(dictionary=True)
    sql = "insert into duplicatedpictures (duPicFileName, srcDir, destDir, action) \
    values (%s, %s, %s, %s)"
    val = (f, srcDir, desDir, 'Delete')
    cursor.execute(sql, val)
    conn.commit()
    conn.close()
    return

def insertDB(f, destDir, srcDir, TT, remark):
    conn = connDB()
    cursor = conn.cursor(dictionary=True)
    sql = "insert into mypicturefiles (pictureFileName, pictureFileDestDir, pictureFileSrcDir, TakeTime, remark) \
    values (%s, %s, %s, %s, %s)"
    val = (f, destDir, srcDir, TT, remark)
    cursor.execute(sql, val)
    conn.commit()
    conn.close()
    return

def selectDB(f):
    conn = connDB()
    #if conn.is_connected():
    #print(True)
    cursor = conn.cursor(dictionary=True)
    sql = "select * from mypicturefiles \
    where pictureFileOldName = %s;"
    val = (f)
    cursor.execute(sql, val)
    #print(cursor)
    result = cursor.fetchone()
    conn.close()
    return result

def selectPathDB(path):
    conn = connDB()
    cursor = conn.cursor(dictionary=True)
    sql = "select * from mypicturefiles \
    where pictureFileSrcDir = %s;"
    val = (path)
    cursor.execute(sql, val)
    #print(cursor)
    result = cursor.fetchall()
    conn.close()
    return result
    
fileToDB()    


In [ ]:
len('20230225_172255_027a91dfa667101abe36030cd10e81971eca8bdf2ac1f0e3b20a2449ba9c8a43.png')

In [ ]:
text= ('D:\사진\2011년\2011년10월\2011년10월01일 BNPP 1N2 Excavation ceremony') 
print(len(text))
len(text.encode())